In [1]:

import contextlib
import duckdb
def duckdb_query(query):
    query = query.replace('"', "'")
    PARAMETERS = 'host=192.20.57.208 user=chen7 port=3306 database=chentiancheng password=1991727asdf'
    ATTACH_QUERY = f"ATTACH '{PARAMETERS}' AS mysqldb (TYPE MYSQL);"
    USE_QUERY = "USE mysqldb;"
    with duckdb.connect() as con:
        con.sql(ATTACH_QUERY)
        # con.sql(USE_QUERY)
        # df = con.sql(query).df()
        df = con.sql(
            f"""SELECT * FROM mysql_query("mysqldb", "{query}")"""
        ).df()
    df_copy = df.copy()
    for col in df_copy.columns:
        with contextlib.suppress(Exception):
            df_copy[col] = df_copy[col].str.decode("utf-8")
    return df_copy

In [2]:
query_sql = """
select 
    TICKER_SYMBOL,
    END_DATE, 
    ADJ_NAV
from 
    fund_adj_nav
where 
    1=1
    and END_DATE between '2023-01-01' and '20241115'
    and TICKER_SYMBOL in ('000001', '163415')
"""
df = duckdb_query(query_sql)
df

,TICKER_SYMBOL,END_DATE,ADJ_NAV
0,000001,2023-01-03,6.563172
1,000001,2023-01-04,6.530872
2,000001,2023-01-05,6.601930
3,000001,2023-01-06,6.589011
4,000001,2023-01-09,6.511493
...,...,...,...
907,163415,2024-11-11,6.114847
908,163415,2024-11-12,6.101082
909,163415,2024-11-13,6.130332
910,163415,2024-11-14,5.970320


In [3]:
import polars as pl
df = pl.from_pandas(df)
df

TICKER_SYMBOL,END_DATE,ADJ_NAV
str,datetime[μs],f64
"""000001""",2023-01-03 00:00:00,6.563172
"""000001""",2023-01-04 00:00:00,6.530872
"""000001""",2023-01-05 00:00:00,6.60193
"""000001""",2023-01-06 00:00:00,6.589011
"""000001""",2023-01-09 00:00:00,6.511493
…,…,…
"""163415""",2024-11-11 00:00:00,6.114847
"""163415""",2024-11-12 00:00:00,6.101082
"""163415""",2024-11-13 00:00:00,6.130332


In [4]:
result = (
    df
    
)

In [5]:
import polars as pl
from datetime import date

a = pl.lit('2020-01-01') 

In [6]:
a

<Expr ['String(2020-01-01)'] at 0x2AE4590F7A0>

In [7]:

cal_period_maxdd(df, '20240101', '20241114')

NameError: name 'cal_period_maxdd' is not defined

In [ ]:
query = """
SELECT
  a.END_DATE,
  a.TYPE_NAME,
  a.RETURN_RATE - b.ChangePCT 
FROM
  fund_derivatives_inner_fund_ret a
  JOIN jy_indexquote b ON a.END_DATE = b.TradingDay 
WHERE
  1 = 1 
  AND a.TYPE_NAME = '主动权益-指数增强-中证500' 
  AND b.SecuCode = '000905'
"""
df = duckdb_query(query)
df

In [ ]:
a = duckdb.sql(
    "SELECT * FROM read_json('d:/config/db_config.json', records='false')"
)
b = a.fetchall()[0][0]

In [ ]:
from quant_utils.db_conn import DB_CONN_JJTG_DATA
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def query_fund_alpha(trade_dt: str) -> pl.DataFrame:
    query_sql = f"""
    SELECT
        END_DATE,
        TICKER_SYMBOL,
        `LEVEL`,
        INDICATOR,
        3M,
        6M,
        1Y,
        2Y,
        3Y
    FROM
        fund_derivatives_fund_alpha_performance 
    WHERE
        1 = 1 
        AND END_DATE = '{trade_dt}' 
    """
    df = DB_CONN_JJTG_DATA.exec_query(query_sql)
    df = pl.from_pandas(df)  # Convert Pandas DataFrame to Polars DataFrame
    df = df.unpivot(
        index=["END_DATE", "TICKER_SYMBOL", "LEVEL", "INDICATOR"],
        on=["3M", "6M", "1Y", "2Y", "3Y"],
        value_name="VALUE",
        variable_name="PERIOD"
    )
    return df

def query_fund_type(trade_dt: str) -> pl.DataFrame:
    query_sql = f"""
    SELECT
        TICKER_SYMBOL,
        LEVEL_1,
        LEVEL_2,
        LEVEL_3
    FROM
        fund_type_own_temp 
    WHERE
        1 = 1 
        AND REPORT_DATE = (
        SELECT
            max( REPORT_DATE ) 
        FROM
            fund_type_own_temp 
        WHERE
            PUBLISH_DATE <= '{trade_dt}'
    )
    """
    df = DB_CONN_JJTG_DATA.exec_query(query_sql)
    df = pl.from_pandas(df)  # Convert Pandas DataFrame to Polars DataFrame
    
    # 检查列是否存在
    if "TICKER_SYMBOL" not in df.columns or "LEVEL_1" not in df.columns or "LEVEL_2" not in df.columns or "LEVEL_3" not in df.columns:
        raise ValueError("Required columns not found in the DataFrame.")
    
    df = df.unpivot(
        index=["TICKER_SYMBOL"],
        on=["LEVEL_1", "LEVEL_2", "LEVEL_3"],
        value_name="LEVEL_NAME",
        variable_name="LEVEL"
    )
    return df

def query_fund_alpha_rank(trade_dt: str) -> pl.DataFrame:
    alpha_perf = query_fund_type(trade_dt).join(
        query_fund_alpha(trade_dt), 
        on=["TICKER_SYMBOL", "LEVEL"]
    )
    
    # Calculate rank
    alpha_perf = alpha_perf.with_columns(
        pl.col("VALUE")
        .over(["LEVEL", "LEVEL_NAME", "INDICATOR", "PERIOD"])
        .rank(method="min", descending=True)
        .alias("VALUE_RANK")
    )
    
    # Calculate percentage rank
    alpha_perf = alpha_perf.with_columns(
        (
            (pl.col("VALUE_RANK")-1) / (pl.col("VALUE_RANK").max()-1)
            .over(["LEVEL", "LEVEL_NAME", "INDICATOR", "PERIOD"])
        )
        .alias("VALUE")
    )
    
    # Adjust VALUE for specific indicators
    condition = pl.col("INDICATOR").is_in(["MAXDD", "ANNUAL_VOL"])
    alpha_perf = alpha_perf.with_columns(
        pl.when(condition)
        .then(1 - pl.col("VALUE"))
        .otherwise(pl.col("VALUE"))
        .alias("VALUE")
    )
    
    # Fill null values
    alpha_perf = alpha_perf.with_columns(
        pl.col("VALUE").fill_null(0.5)
    )
    
    return alpha_perf

def query_fund_alpha_score(trade_dt: str) -> pl.DataFrame:
    alpha_perf_rank = query_fund_alpha_rank(trade_dt)
    indicator_weight = {
        "CUM_ALPHA": 0.35,
        "IR": 0.35,
        "MAXDD": 0.3,
        "ANNUAL_VOL": 0.0,
    }
    period_weight = {
        "3M": 0.2,
        "6M": 0.2,
        '1Y': 0.2,
        '2Y': 0.2,
        '3Y': 0.2,
    }
    
    # Initialize INDICATOR_WEIGHT and PERIOD_WEIGHT columns
    alpha_perf_rank = alpha_perf_rank.with_columns(
        pl.lit(0.0).alias("INDICATOR_WEIGHT"),
        pl.lit(0.0).alias("PERIOD_WEIGHT")
    )
    
    # Apply indicator weights
    for indicator, weight in indicator_weight.items():
        alpha_perf_rank = alpha_perf_rank.with_columns(
            pl.when(pl.col("INDICATOR") == indicator)
            .then(weight)
            .otherwise(pl.col("INDICATOR_WEIGHT"))
            .alias("INDICATOR_WEIGHT")
        )
    
    # Apply period weights
    for period, weight in period_weight.items():
        alpha_perf_rank = alpha_perf_rank.with_columns(
            pl.when(pl.col("PERIOD") == period)
            .then(weight)
            .otherwise(pl.col("PERIOD_WEIGHT"))
            .alias("PERIOD_WEIGHT")
        )
    
    # Calculate weighted value
    alpha_perf_rank = alpha_perf_rank.with_columns(
        pl.col("VALUE") * pl.col("INDICATOR_WEIGHT") * pl.col("PERIOD_WEIGHT")
    )
    # Aggregate by TICKER_SYMBOL, LEVEL, LEVEL_NAME
    alpha_perf_score = (
        alpha_perf_rank
        .group_by(["TICKER_SYMBOL", "LEVEL", "LEVEL_NAME"])
        .agg(pl.col("VALUE").sum().alias("VALUE"))
        .with_columns(pl.col("VALUE").fill_null(np.nan))
    )
    
    return alpha_perf_score

def query_fund_alpha_score_rank(trade_dt: str) -> pl.DataFrame:
    alpha_perf_score = query_fund_alpha_score(trade_dt)
    
    # Calculate rank
    alpha_perf_score = alpha_perf_score.with_columns(
        pl.col("VALUE")
        .over(["LEVEL", "LEVEL_NAME"])
        .rank(method="min", descending=True)
        .alias("RANK")
    )
    
    # Calculate percentage rank
    alpha_perf_score = alpha_perf_score.with_columns(
        (pl.col("RANK") / pl.col("RANK").max().over(["LEVEL", "LEVEL_NAME"]))
        .alias("RANK")
    )
    
    # Multiply rank by 100
    alpha_perf_score = alpha_perf_score.with_columns(
        pl.col("RANK") * 100
    )
    
    return alpha_perf_score

df = query_fund_alpha_score_rank("20241114")

In [ ]:
import os
import pandas as pd
from dateutil.parser import parse
import polars as pl
import duckdb
import numpy as np
ftr_path = "F:/data_ftr/fund_nav/"
ftr_list = os.listdir(ftr_path)
parquet_path = "F:/data_parquet/fund_nav/"

In [ ]:
def get_fund_nav_by_parquet(
        start_date: str, 
        end_date: str,
        parquet_path: str = "F:/data_parquet/fund_nav/"
) -> pl.DataFrame:
    start_date = parse(start_date).strftime("%Y-%m-%d")
    end_date = parse(end_date).strftime("%Y-%m-%d")
    query = f"""
        SELECT 
            END_DATE, 
            TICKER_SYMBOL, 
            ADJ_NAV
        FROM 
            '{parquet_path}*.parquet' 
        where 
            1=1
            and END_DATE between '{start_date}' and '{end_date}'
        order by
            END_DATE,
            TICKER_SYMBOL
    """
    with duckdb.connect() as con:
        df = con.sql(query).pl()
    return df

def get_trade_cal_by_parquet(
        parquet_path: str = "F:/data_parquet/"
) -> pl.DataFrame:
    query = f"""
        SELECT
            *
        FROM
            '{parquet_path}trade_cal.parquet'
    """
    with duckdb.connect() as con:
        df = con.sql(query).pl()
    return df

In [ ]:

def filter_fund_nav(df:pl.DataFrame, start_date:str, end_date:str):
    if isinstance(start_date, str):
        start_date = parse(start_date)
    if isinstance(end_date, str):
        end_date = parse(end_date)

    df_filtered = (
        df.lazy()
        .filter(pl.col("END_DATE").is_between(start_date, end_date))
        .with_columns(
            [
                pl.col("END_DATE").min().over("TICKER_SYMBOL").alias("START_DATE"),
                pl.col("END_DATE").max().over("TICKER_SYMBOL").alias("MAX_DATE"),
            ]
        )
        .filter(
            (pl.col("START_DATE") == start_date)&(pl.col("MAX_DATE") == end_date)
        )
    ).collect()
    # 寻找开始之后的第一个日期
    trade_cal = (
        get_trade_cal_by_parquet().filter(
            (pl.col("PREV_TRADE_DATE") == start_date)
            &(pl.col("IF_TRADING_DAY") == 1)
        ).select(
            pl.col("PREV_TRADE_DATE").alias("START_DATE"),
            pl.col("TRADE_DT").alias("OPERATE_DATE"),
        )
    )
    df_filtered = df_filtered.join(trade_cal, on="START_DATE", how="left")
    df_filtered = (
        df_filtered.lazy()
        .select(
            [
                pl.all().exclude(["OPERATE_DATE", "MAX_DATE"]),
                ((pl.col("END_DATE") - pl.col("OPERATE_DATE")).dt.total_seconds() / (60 * 60 * 24) + 1).alias("OPERATION_DAYS")
            ]
        )
        .with_columns(
            pl.when(pl.col("OPERATION_DAYS") < 0).then(0).otherwise(pl.col("OPERATION_DAYS")).alias("OPERATION_DAYS")
        )

    ).collect()
    return df_filtered

def prepare_dates(start_date, end_date):
    if isinstance(start_date, str):
        start_date = parse(start_date)
    if isinstance(end_date, str):
        end_date = parse(end_date)
    return start_date, end_date


def cal_period_drawdown(df:pl.DataFrame, start_date:str, end_date:str):

    start_date, end_date = prepare_dates(start_date, end_date)
    
    # df_filtered = filter_fund_nav(df, start_date, end_date)
    temp_df = (
        df.lazy()
        # 计算累计最大值
        .with_columns(
            [
                (100*(1 - pl.col("ADJ_NAV") / _cal_cum_max(col_name="ADJ_NAV", over_by="TICKER_SYMBOL", order_by="END_DATE"))).alias("drawdown")
            ]
        )
        # 计算滚动最大回撤
        .select(
            [
                pl.col("TICKER_SYMBOL"),
                pl.col("START_DATE"),
                pl.col("END_DATE"),
                pl.col("drawdown"),
                _cal_cum_max(
                    col_name="drawdown", 
                    over_by="TICKER_SYMBOL", 
                    order_by="END_DATE"
                ).alias("MAX_DRAWDOWN"),
            ]
        )
    )
    return temp_df.collect()

def cal_period_maxdrawdown(drawdown_df:pl.DataFrame, start_date:str, end_date:str):
    start_date, end_date = prepare_dates(start_date, end_date)
    return (
        drawdown_df
        .filter(pl.col("END_DATE") == end_date)
        .select(
            [
                pl.col("TICKER_SYMBOL"), 
                pl.col("START_DATE"),
                pl.col("END_DATE"),
                pl.col("MAX_DRAWDOWN")
            ]
        )
    )

def cal_period_maxdd_date_and_recover(drawdown_df:pl.DataFrame):
    dd_date = (
        # 筛选回撤等于最大回撤的日期并且最大回撤不为0
        drawdown_df.filter(
            (pl.col('drawdown') == pl.col('MAX_DRAWDOWN'))
            & (pl.col('MAX_DRAWDOWN') != 0)
        )
        # 只保留最大回撤日期
        .select(
            pl.col('TICKER_SYMBOL'),
            pl.col('END_DATE').alias('MAXDD_DATE'),
            pl.col('END_DATE')
            .max()
            .over('TICKER_SYMBOL')
            .alias('MAX_DATE_TEMP'),
        )
        .filter(pl.col('MAX_DATE_TEMP') == pl.col('MAXDD_DATE'))
        .select(pl.col('TICKER_SYMBOL'), pl.col('MAXDD_DATE'))
    )
    dd_recover = (
        # 回撤与回撤修复并表
        drawdown_df.join(dd_date, on=['TICKER_SYMBOL'], how='left')
        # 筛选大于最大回撤日期并且当前回撤为0的日期
        .filter(
            (pl.col("END_DATE") >= pl.col("MAXDD_DATE"))
            &(pl.col("drawdown") ==0)
        )
        # 计算回撤为0的日期中最小的回撤修复日期
        .with_columns(
            [
                pl.col('END_DATE').min().over('TICKER_SYMBOL').alias('MIN_DD_RECOVERY_DATE')
            ]
        )
        # 筛选出最小的回撤修复日期
        .filter(
            (pl.col("END_DATE") == pl.col("MIN_DD_RECOVERY_DATE"))
        )
        # 计算最大回撤修复天数
        .select(
            pl.col("TICKER_SYMBOL"),
            pl.col('MIN_DD_RECOVERY_DATE').alias('RECOVER_DATE'),
            ((pl.col('MIN_DD_RECOVERY_DATE') - pl.col('MAXDD_DATE')).dt.total_seconds() / (60 * 60 * 24)).alias('MAXDD_RECOVER'),
        )
    )
    return dd_date.join(dd_recover, on=['TICKER_SYMBOL'], how='left').fill_null(99999)

def _cal_cum_max(
        col_name:str = "ADJ_NAV", 
        over_by:str = "TICKER_SYMBOL", 
        order_by:str = "END_DATE"
    ):
    return pl.col(col_name).cum_max().over(over_by, order_by=pl.col(order_by))
def _cal_ret(
    col_name:str = "AJD_NAV",
    over_by:str = "TICKER_SYMBOL",
    order_by:str = "END_DATE"
):
    return (
        (pl.col(col_name).last() / pl.col(col_name).first() - 1)*100
    ).over(over_by, order_by=pl.col(order_by)).alias("CUM_RETURN")

def _cal_daily_return(
    col_name:str = "AJD_NAV",
    over_by:str = "TICKER_SYMBOL",
    order_by:str = "END_DATE"
):
    return (
        pl.col(col_name)/pl.col(col_name).shift(1) - 1 
    ).over(over_by, order_by=pl.col(order_by)).alias("DAILY_RETURN")

def _cal_vol(
    col_name:str = "DAILY_RETURN",
    over_by:str = "TICKER_SYMBOL",
    order_by:str = "END_DATE"
):
    return pl.col(col_name).std().over(over_by, order_by=pl.col(order_by)).alias("VOLATILITY")
def _cal_cal_period_ret_and_vol(df:pl.DataFrame, start_date:str, end_date:str):
    start_date, end_date = prepare_dates(start_date, end_date)

    return (
        df.lazy()
        # 计算每日收益率和累计收益率
        .with_columns(
            [_cal_daily_return(col_name="ADJ_NAV").alias("DAILY_RETURN")]
        )
        # 计算累计收益率和波动率
        .with_columns(
            [
                _cal_vol(col_name="DAILY_RETURN").alias("VOLATILITY"),
                _cal_ret(col_name="ADJ_NAV").alias("CUM_RETURN"),
            ]
        ).filter(pl.col("END_DATE") == end_date)
    ).collect()

def _cal_annual_return_and_vol(df:pl.DataFrame):
    return (
        df.select(
                [
                    pl.col("TICKER_SYMBOL"),
                    pl.col("START_DATE"),
                    pl.col("END_DATE"),
                    pl.col("CUM_RETURN"),
                    (((pl.col("CUM_RETURN")/100 + 1)**(365/pl.col("OPERATION_DAYS")) - 1)*100).alias("ANNUAL_RETURN"),
                    pl.col("VOLATILITY"),
                    (pl.col("VOLATILITY")*np.sqrt(252)*100).alias("ANNUAL_VOLATILITY")
                ]
            )
    )
def cal_period_ret_and_vol(df:pl.DataFrame, start_date:str, end_date:str):
    start_date, end_date = prepare_dates(start_date, end_date)
    result = (
            _cal_cal_period_ret_and_vol(df, start_date, end_date)
            .filter(pl.col("END_DATE") == end_date)
    )
    return _cal_annual_return_and_vol(result)

def cal_period_perform(df:pl.DataFrame, start_date:str, end_date:str):
    start_date, end_date = prepare_dates(start_date, end_date)
    df_filtered = filter_fund_nav(df, start_date, end_date)
    temp_df = cal_period_ret_and_vol(df_filtered, start_date, end_date)
    dd = cal_period_drawdown(df_filtered, start_date, end_date)
    dd_date_recover = cal_period_maxdd_date_and_recover(dd)
    maxdd = cal_period_maxdrawdown(dd, '20221230', '20241114')
    maxdd = maxdd.join(dd_date_recover, on=['TICKER_SYMBOL'], how='left')
    temp_df = (
        temp_df
        .join(maxdd, on=['TICKER_SYMBOL', "START_DATE", "END_DATE"], how='left')
        .with_columns(
            [
                (pl.col("CUM_RETURN")/pl.col("VOLATILITY")/100).alias("SHARP_RATIO"),
                (pl.col("ANNUAL_RETURN")/pl.col("ANNUAL_VOLATILITY")).alias("SHARP_RATIO_ANNUAL"),
                (pl.col("ANNUAL_RETURN")/pl.col("MAX_DRAWDOWN")).alias("CALMAR_RATIO_ANNUAL"),
            ]
        )
    )
    return temp_df


In [ ]:
df = get_fund_nav_by_parquet('20231228', '20241114')

In [ ]:
filter_fund_nav(df, '20231229', '20241114')

END_DATE,TICKER_SYMBOL,ADJ_NAV,START_DATE,OPERATION_DAYS
date,str,f64,date,f64
2023-12-29,"""000001""",5.05424,2023-12-29,0.0
2023-12-29,"""000003""",0.944113,2023-12-29,0.0
2023-12-29,"""000004""",0.928828,2023-12-29,0.0
2023-12-29,"""000005""",1.613538,2023-12-29,0.0
2023-12-29,"""000006""",2.387004,2023-12-29,0.0
…,…,…,…,…
2024-11-14,"""970210""",1.0285,2023-12-29,318.0
2024-11-14,"""970211""",1.1124,2023-12-29,318.0
2024-11-14,"""970212""",1.1094,2023-12-29,318.0


In [ ]:
from quant_utils.db_conn import DB_CONN_JJTG_DATA
import pandas as pd
def get_needed_dates_df(end_date: str = None, start_date: str = None) -> pd.DataFrame:
    query_sql = f"""
    SELECT
        date_format(START_DATE,'%Y%m%d') as START_DATE,
        date_format(END_DATE,'%Y%m%d') as END_DATE,
        DATE_NAME
    FROM
        `portfolio_dates`
    WHERE
        1=1
        and (END_DATE BETWEEN '{start_date}' AND '{end_date}')
    """
    return DB_CONN_JJTG_DATA.exec_query(query_sql)


In [8]:
import polars as pl
from dateutil.parser import parse
from quant_pl.performance_pl import PerformancePL
from quant_utils.db_conn import DB_CONN_JJTG_DATA
import pandas as pd
def get_nav(end_date):
    query_sql = f"""
    SELECT
        DATE_FORMAT( TRADE_DT, "%Y%m%d" ) AS END_DATE,
        a.PORTFOLIO_NAME AS TICKER_SYMBOL,
        PORTFOLIO_NAV AS NAV 
    FROM
        portfolio_nav a
        JOIN portfolio_info b ON a.PORTFOLIO_NAME = b.PORTFOLIO_NAME 
    WHERE
        1 = 1 
        AND a.TRADE_DT >= b.LISTED_DATE
        AND a.TRADE_DT <= '{end_date}'
    """
    return DB_CONN_JJTG_DATA.exec_query(query_sql)

In [9]:
df = get_nav("20241120")
df = pl.from_pandas(df).with_columns(pl.col("END_DATE").str.strptime(pl.Datetime, "%Y%m%d"))

In [10]:
start_date = '20210903'
end_date = '20241120'
perf = PerformancePL(df, start_date, end_date)
perf.stats()

TICKER_SYMBOL,START_DATE,END_DATE,CUM_RETURN,ANNUAL_RETURN,VOLATILITY,ANNUAL_VOLATILITY,SHARP_RATIO,SHARP_RATIO_ANNUAL,MAXDD,CALMAR_RATIO_ANNUAL,MAXDD_DATE,MAXDD_RECOVER,RECOVER_DATE
str,datetime[μs],datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,datetime[μs],f64,datetime[μs]
"""知己优选-均衡小确幸""",2021-09-03 00:00:00,2024-11-20 00:00:00,-2.505698,-0.787187,0.55475,8.806385,-4.516804,-0.089388,17.159819,-0.045874,2024-02-05 00:00:00,99999.0,null
"""知己优选-进取全明星""",2021-09-03 00:00:00,2024-11-20 00:00:00,-19.755698,-6.624834,1.037303,16.466672,-19.045257,-0.402318,39.573375,-0.167406,2024-02-05 00:00:00,99999.0,null
